In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-18 15:53:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-03-18 15:53:32 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

Deliverable 2: Determine Bias of Vine Reviews

In [7]:
# filter the dataframe to retrieve all rows where total_votes >= 20
filtered_df = df[(df["total_votes"] >= 20)]
filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [8]:
# Check the data types
filtered_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [9]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = filtered_df[(filtered_df["helpful_votes"] / filtered_df["total_votes"] >= 0.5)]
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [10]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
yes_vine_df = helpful_votes_df[(helpful_votes_df["vine"] == 'Y')]
yes_vine_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+-------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'.
no_vine_df = helpful_votes_df[(helpful_votes_df["vine"] == 'N')]
no_vine_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
# Determine the total number of reviews with vines
total_reviews_with_vines = yes_vine_df.count()
print(total_reviews_with_vines)

107


In [20]:
# Determine the percentage of 5 star reviews for reviews with vines
fivestars_with_vines = yes_vine_df[(yes_vine_df["star_rating"] == 5)].count()
print(fivestars_with_vines)

56


In [21]:
# Determine the percentage of 5 star reviews for reviews with vines
percent_fivestars_w_vines = (fivestars_with_vines / total_reviews_with_vines) * 100
percent_fivestars_w_vines = "{:.2f}%".format(percent_fivestars_w_vines)
print(percent_fivestars_w_vines)

52.34%


In [22]:
# Determine the total number of reviews without vines
total_reviews_without_vines = no_vine_df.count()
print(total_reviews_without_vines)

39869


In [23]:
# Determine the total number of reviews without vines
total_reviews_without_vines = no_vine_df.count()
print(total_reviews_without_vines)

39869


In [24]:
# Determine the percentage of 5 star reviews for reviews with vines
fivestars_without_vines = no_vine_df[(no_vine_df["star_rating"] == 5)].count()
print(fivestars_without_vines)

21005


In [25]:
# Determine the percentage of 5 star reviews for reviews with vines
percent_fivestars_wo_vines = (fivestars_without_vines / total_reviews_without_vines) * 100
percent_fivestars_wo_vines = "{:.2f}%".format(percent_fivestars_wo_vines)
print(percent_fivestars_wo_vines)

52.69%


In [26]:
# Create a DataFrame with the Results
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

vine_data = [("Total Reviews", total_reviews_with_vines, total_reviews_without_vines),
        ("Five Star Reviews", fivestars_with_vines, fivestars_without_vines),
        ("Percent Five Star Reviews", percent_fivestars_w_vines, percent_fivestars_wo_vines)
        ]

schema = StructType([
                StructField(" ", StringType(), True),
                StructField("Reviews with Vines", StringType(), True),
                StructField("Reviews without Vines", StringType(), True)
])

vine_review_df = spark.createDataFrame(data=vine_data, schema=schema)

vine_review_df.show(truncate=False)

+-------------------------+------------------+---------------------+
|                         |Reviews with Vines|Reviews without Vines|
+-------------------------+------------------+---------------------+
|Total Reviews            |107               |39869                |
|Five Star Reviews        |56                |21005                |
|Percent Five Star Reviews|52.34%            |52.69%               |
+-------------------------+------------------+---------------------+



In [27]:
# Determine the nubmer of vine verified purchases
vine_verified = yes_vine_df[(yes_vine_df['verified_purchase']=='Y')].count() 
percent_vine_verified = vine_verified / total_reviews_with_vines * 100
percent_vine_verified = "{:.2f}%".format(percent_vine_verified)

print(vine_verified, percent_vine_verified)

2 1.87%


In [31]:
# Determine the nubmer of vine verified purchases
no_vine_verified = no_vine_df[(no_vine_df['verified_purchase']=='Y')].count() 
percent_no_vine_verified = no_vine_verified / total_reviews_without_vines * 100
percent_no_vine_verified = "{:.2f}%".format(percent_no_vine_verified)

print(no_vine_verified, percent_no_vine_verified)

28068 70.40%


In [32]:
# Create a DataFrame with the Results
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

vine_data = [("Total Reviews", total_reviews_with_vines, total_reviews_without_vines),
        ("Verified Purchases", vine_verified, no_vine_verified),
        ("Percent Verified Purchases", percent_vine_verified, percent_no_vine_verified)
        ]

schema = StructType([
                StructField(" ", StringType(), True),
                StructField("Reviews with Vines", StringType(), True),
                StructField("Reviews without Vines", StringType(), True)
])

verified_purchases_df = spark.createDataFrame(data=vine_data, schema=schema)

verified_purchases_df.show(truncate=False)

+--------------------------+------------------+---------------------+
|                          |Reviews with Vines|Reviews without Vines|
+--------------------------+------------------+---------------------+
|Total Reviews             |107               |39869                |
|Verified Purchases        |2                 |28068                |
|Percent Verified Purchases|1.87%             |70.40%               |
+--------------------------+------------------+---------------------+



In [42]:
# Average star rating
avg_rating_df = filtered_df.groupBy('vine').mean()
avg_rating_df = avg_rating_df[["vine","avg(star_rating)"]]
avg_rating_df.show()

+----+-----------------+
|vine| avg(star_rating)|
+----+-----------------+
|   Y|4.290909090909091|
|   N|3.738657279587705|
+----+-----------------+

